# Winning Jeopardy

This is a project that attempts to use hypothesis testing---chi-squared hypothesis testing in particular---to work out whether or not certain there are any patterns in Jeopardy questions.

None of the chi-square tests (see below) don't report an p-value of $\alpha < 0.05$, so we can safely conclude that there is no statistically significant relationship in my attempts to find patterns in how Jeopardy questions are presented.

In [30]:
# Import and configure the necessary stuff
import re
import random
import numpy as np
import pandas as pd

from scipy.stats import chisquare

jeopardy = pd.read_csv("jeopardy.csv")

jeopardy

Show Number    Air Date             Round  \
0             4680  2004-12-31         Jeopardy!   
1             4680  2004-12-31         Jeopardy!   
2             4680  2004-12-31         Jeopardy!   
3             4680  2004-12-31         Jeopardy!   
4             4680  2004-12-31         Jeopardy!   
5             4680  2004-12-31         Jeopardy!   
6             4680  2004-12-31         Jeopardy!   
7             4680  2004-12-31         Jeopardy!   
8             4680  2004-12-31         Jeopardy!   
9             4680  2004-12-31         Jeopardy!   
10            4680  2004-12-31         Jeopardy!   
11            4680  2004-12-31         Jeopardy!   
12            4680  2004-12-31         Jeopardy!   
13            4680  2004-12-31         Jeopardy!   
14            4680  2004-12-31         Jeopardy!   
15            4680  2004-12-31         Jeopardy!   
16            4680  2004-12-31         Jeopardy!   
17            4680  2004-12-31         Jeopardy!   
18            4680  2004-12-31         Jeopardy!   
19            4680  2004-12-31         Jeopardy!   
20            4680  2004-12-31         Jeopardy!   
21            4680  2004-12-31         Jeopardy!   
22            4680  2004-12-31         Jeopardy!   
23            4680  2004-12-31         Jeopardy!   
24            4680  2004-12-31         Jeopardy!   
25            4680  2004-12-31         Jeopardy!   
26            4680  2004-12-31         Jeopardy!   
27            4680  2004-12-31         Jeopardy!   
28            4680  2004-12-31         Jeopardy!   
29            4680  2004-12-31  Double Jeopardy!   
...            ...         ...               ...   
19969         5694  2009-05-14  Double Jeopardy!   
19970         5694  2009-05-14  Double Jeopardy!   
19971         5694  2009-05-14  Double Jeopardy!   
19972         5694  2009-05-14  Double Jeopardy!   
19973         5694  2009-05-14  Double Jeopardy!   
19974         5694  2009-05-14  Double Jeopardy!   
19975         5694  2009-05-14  Double Jeopardy!   
19976         5694  2009-05-14  Double Jeopardy!   
19977         5694  2009-05-14  Double Jeopardy!   
19978         5694  2009-05-14  Double Jeopardy!   
19979         5694  2009-05-14  Double Jeopardy!   
19980         5694  2009-05-14  Double Jeopardy!   
19981         5694  2009-05-14  Double Jeopardy!   
19982         5694  2009-05-14  Double Jeopardy!   
19983         5694  2009-05-14  Double Jeopardy!   
19984         5694  2009-05-14  Double Jeopardy!   
19985         5694  2009-05-14  Double Jeopardy!   
19986         5694  2009-05-14  Double Jeopardy!   
19987         5694  2009-05-14   Final Jeopardy!   
19988         3582  2000-03-14         Jeopardy!   
19989         3582  2000-03-14         Jeopardy!   
19990         3582  2000-03-14         Jeopardy!   
19991         3582  2000-03-14         Jeopardy!   
19992         3582  2000-03-14         Jeopardy!   
19993         3582  2000-03-14         Jeopardy!   
19994         3582  2000-03-14         Jeopardy!   
19995         3582  2000-03-14         Jeopardy!   
19996         3582  2000-03-14         Jeopardy!   
19997         3582  2000-03-14         Jeopardy!   
19998         3582  2000-03-14         Jeopardy!   

                                Category   Value  \
0                                HISTORY    $200   
1        ESPN's TOP 10 ALL-TIME ATHLETES    $200   
2            EVERYBODY TALKS ABOUT IT...    $200   
3                       THE COMPANY LINE    $200   
4                    EPITAPHS & TRIBUTES    $200   
5                         3-LETTER WORDS    $200   
6                                HISTORY    $400   
7        ESPN's TOP 10 ALL-TIME ATHLETES    $400   
8            EVERYBODY TALKS ABOUT IT...    $400   
9                       THE COMPANY LINE    $400   
10                   EPITAPHS & TRIBUTES    $400   
11                        3-LETTER WORDS    $400   
12                               HISTORY    $600   
13       ESPN's TOP 10 ALL-TIME ATHLETES    $600   
14

In [31]:
# Define functons to format the text
def normalize_text(string):
    string = string.lower()
    string = re.sub("[^A-Za-z0-9\s]", "", string)
    return string

def normalize_values(string):
    string = re.sub("[^A-Za-z0-9\s]", "", string)
    try:
        string = int(string)
    except Exception:
        string = 0
    return string

In [32]:
jeopardy.columns = ['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question', 'Answer']

jeopardy["clean_question"] = jeopardy["Question"].apply(normalize_text)
jeopardy["clean_answer"] = jeopardy["Answer"].apply(normalize_text)
jeopardy["clean_value"] = jeopardy["Value"].apply(normalize_values)

In [33]:
jeopardy["Air Date"] = pd.to_datetime(jeopardy["Air Date"])
jeopardy.dtypes

Show Number                int64
Air Date          datetime64[ns]
Round                     object
Category                  object
Value                     object
Question                  object
Answer                    object
clean_question            object
clean_answer              object
clean_value                int64
dtype: object

In [34]:
def count_matches(row):
    split_answer = row["clean_answer"].split(" ")
    split_question = row["clean_question"].split(" ")
    if "the" in split_answer:
        split_answer.remove("the")
    if len(split_answer) == 0:
        return 0
    match_count = 0
    for item in split_answer:
        if item in split_question:
            match_count += 1
    return match_count / len(split_answer)

jeopardy["answer_in_question"] = jeopardy.apply(count_matches, axis=1)
avg_ansinquestion = jeopardy["answer_in_question"].mean()

question_overlap = []
terms_used = set()

avg_ansinquestion

0.06049325706933587

In [35]:
qo, tuse = [], set()
jeopardy = jeopardy.sort_values("Air Date")

for n, k in jeopardy.iterrows():
    split_question = k["clean_question"].split(" ")
    split_question = [k for k in split_question if len(k) > 5]
    matches = 0
    
    for w in split_question:
        if w in tuse:
            matches = matches + 1
    for w in split_question:
        tuse.add(w)
    if len(split_question) > 0:
        matches = matches / len(split_question)
    qo.append(matches)
    
jeopardy["question_overlap"] = qo
question_overlap_mean = jeopardy["question_overlap"].mean()
question_overlap_mean

0.6876260592169802

In [36]:
# Low values vs. high value questions

def detval(r):
    if r["clean_value"] > 800:
        return 1
    return 0

jeopardy["high_value"] = jeopardy.apply(detval, axis=1)

In [37]:
def count_usage(t):
    l, h = 0, 0
    for n, k in jeopardy.iterrows():
        if t in k["clean_question"].split(" "):
            if k["high_value"] == 1:
                h = h + 1
            else:
                l = l + 1
    return h, l

In [41]:
used_terms = list(tuse)
ct = [random.choice(used_terms) for k in range(10)]

expected_obs = []

for k in ct:
    expected_obs.append(count_usage(k))
    
expected_obs

[(0, 1),
 (0, 1),
 (0, 1),
 (1, 0),
 (0, 1),
 (0, 1),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 2)]

In [45]:
count_hv = jeopardy[jeopardy["high_value"] == 1].shape[0]
count_lv = jeopardy[jeopardy["high_value"] == 0].shape[0]

chi_squared = []
for observation in expected_obs:
    total = sum(observation)
    total_prop = total / jeopardy.shape[0]
    hv_exp = total_prop * count_hv
    lv_exp = total_prop * count_lv
    
    o = np.array([observation[0], observation[1]])
    e = np.array([hv_exp, lv_exp])
    chi_squared.append(chisquare(o, e))
    
chi_squared

[Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=2.487792117195675, pvalue=0.11473257634454047),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=2.487792117195675, pvalue=0.11473257634454047),
 Power_divergenceResult(statistic=0.401962846126884, pvalue=0.5260772985705469),
 Power_divergenceResult(statistic=0.03188116723440362, pvalue=0.8582887163235293)]